In [28]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [29]:
# reading the dataset and creating a copy of the dataset, which is then used to create non-ambient dataset

one = pd.read_csv('S1.csv')
onen=one.copy()
one['Locomotion'] = one[['Locomotion', 'ML_Both_Arms']].max(axis=1)
one.drop(['LL_Left_Arm_Object', 'LL_Right_Arm_Object', 'ML_Both_Arms'], axis=1, inplace=True)
onen.drop(['LL_Left_Arm_Object', 'LL_Right_Arm_Object', 'ML_Both_Arms'], axis=1, inplace=True)

two = pd.read_csv('S2.csv')
twon=two.copy()
two['Locomotion'] = two[['Locomotion', 'ML_Both_Arms']].max(axis=1)
two.drop(['LL_Left_Arm_Object', 'LL_Right_Arm_Object', 'ML_Both_Arms'], axis=1, inplace=True)
twon.drop(['LL_Left_Arm_Object', 'LL_Right_Arm_Object', 'ML_Both_Arms'], axis=1, inplace=True)

three = pd.read_csv('S3.csv')
threen=three.copy()
three['Locomotion'] = three[['Locomotion', 'ML_Both_Arms']].max(axis=1)
three.drop(['LL_Left_Arm_Object', 'LL_Right_Arm_Object', 'ML_Both_Arms'], axis=1, inplace=True)
threen.drop(['LL_Left_Arm_Object', 'LL_Right_Arm_Object', 'ML_Both_Arms'], axis=1, inplace=True)

four = pd.read_csv('S4.csv')
fourn=four.copy()
four['Locomotion'] = four[['Locomotion', 'ML_Both_Arms']].max(axis=1)
four.drop(['LL_Left_Arm_Object', 'LL_Right_Arm_Object', 'ML_Both_Arms'], axis=1, inplace=True)
fourn.drop(['LL_Left_Arm_Object', 'LL_Right_Arm_Object', 'ML_Both_Arms'], axis=1, inplace=True)

In [30]:
# using one as validation

def base_model(train_label, test_label,):

    train = pd.concat(train_label)
    test = test_label

    X_train = train.drop('Locomotion', axis=1)
    y_train = train['Locomotion']

    X_test = test.drop('Locomotion', axis=1)
    y_test = test['Locomotion']

    # standardizing the data
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


    # using the naive bayes classifier

    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import accuracy_score

    model = GaussianNB()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # print model metrics
    from sklearn.metrics import classification_report
    from sklearn.metrics import precision_score, recall_score, f1_score

    return({'accuracy':accuracy_score(y_test, y_pred), 'Precision': precision_score(y_test, y_pred, average='weighted'), 'Recall': recall_score(y_test, y_pred, average='weighted'), 'f1_score': f1_score(y_test, y_pred, average='weighted'),'Macro Average':precision_score(y_test, y_pred, average='macro')})

In [31]:
temp = {'one': base_model([twon, threen, fourn], one), 'two': base_model([onen, threen, fourn], two), 'three': base_model([onen, twon, fourn], three), 'four': base_model([onen, twon, threen], four)
        }
convert = pd.DataFrame(temp)

# convert rows to columns and columns to rows
convert = convert.T
# a separate column for the index
convert['Validation Set'] = convert.index
# reset the index
convert.reset_index(drop=True, inplace=True)
convert = convert[['Validation Set', 'accuracy', 'Precision', 'Recall', 'f1_score', 'Macro Average']]
print('Without Ambient Data')
convert.head()

# save the results to a csv file
convert.to_csv('reports/NaiveBayes-without.csv', index=False)

Without Ambient Data


,Validation Set,accuracy,Precision,Recall,f1_score,Macro Average
0,one,0.618594,0.525248,0.618594,0.567212,0.364402
1,two,0.611444,0.520833,0.611444,0.558277,0.329848
2,three,0.449080,0.358765,0.449080,0.396130,0.176508
3,four,0.580524,0.491905,0.580524,0.531239,0.325793


In [32]:
temp = {'one': base_model([two, three, four], one), 'two': base_model([one, three, four], two), 'three': base_model([one, two, four], three), 'four': base_model([one, two, three], four)
        }
convert = pd.DataFrame(temp)

# convert rows to columns and columns to rows
convert = convert.T
# a separate column for the index
convert['Validation Set'] = convert.index
# reset the index
convert.reset_index(drop=True, inplace=True)
convert = convert[['Validation Set', 'accuracy', 'Precision', 'Recall', 'f1_score', 'Macro Average']]
print('With Ambient Data')
convert.head()

# save data to csv
convert.to_csv('reports/NaiveBayes-with.csv', index=False)

With Ambient Data


,Validation Set,accuracy,Precision,Recall,f1_score,Macro Average
0,one,0.689265,0.705952,0.689265,0.685551,0.697218
1,two,0.650776,0.663101,0.650776,0.645197,0.581922
2,three,0.504703,0.477974,0.504703,0.476329,0.436306
3,four,0.631492,0.659262,0.631492,0.631588,0.569525


## Save both Datasets

In [33]:
train = pd.concat([twon, threen, fourn])
train['Locomotion'] = train['Locomotion'].replace([1, 2, 5, 4], [1, 2, 5, 4])

train_a = pd.concat([two, three, four])
train_a['Locomotion'] = train_a['Locomotion'].replace([406516, 406517, 404516, 404517], [6, 7, 8, 9])
test = one
test['Locomotion'] = test['Locomotion'].replace([406516, 406517, 404516, 404517], [6, 7, 8, 9])

In [34]:
train.drop(['Accelerometer DOOR1 accX',
       'Accelerometer DOOR1 accY', 'Accelerometer DOOR1 accZ',
       'Accelerometer DOOR2 accX', 'Accelerometer DOOR2 accY',
       'Accelerometer DOOR2 accZ'], axis=1, inplace=True)
train.to_csv('train-n.csv', index=False)
train_a.to_csv('train-a.csv', index=False)
test.to_csv('test.csv', index=False)